In [1]:
import numpy as np
import networkx as nx
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.sampling import BayesianModelSampling
from scipy.stats import beta, dirichlet
from operator import add
import matplotlib.pyplot as plt

## Variables

In [27]:
variables=list()
variables=['Enfermedad','Tratamiento','Reaccion','Final']

In [ ]:
## Parametros

In [476]:
alpha=0.75

## Orden

In [28]:
orden=list()
orden=[['Tratamiento','Enfermedad'],
       ['Reaccion','Tratamiento'],
       ['Reaccion','Enfermedad'],
       ['Final','Enfermedad'],
       ['Final','Tratamiento'],
       ['Final','Reaccion']]

## True Causal Model From Environment:

In [622]:
model = BayesianModel([('Enfermedad', 'Final'), ('Tratamiento', 'Final'), ('Tratamiento', 'Reaccion'),
                       ('Reaccion', 'Final')])

cpd_e = TabularCPD(variable='Enfermedad', variable_card=2, values=[[0.7, 0.3]])
cpd_tr = TabularCPD(variable='Tratamiento', variable_card=2, values=[[0.5, 0.5]])

cpd_r_tr = TabularCPD(variable='Reaccion', variable_card=2, 
                   values=[[.7,.4], [.3,.6]], #vivir|farmaco vivir|cirugia #morir|farmaco morir|cirugia
                   evidence=['Tratamiento'],
                   evidence_card=[2])

cpd_f_e_tr_r = TabularCPD(variable='Final', variable_card=2, 
                   values=[[.6,0,.8,0,.4,0,.9,0],
                           [.4,1,.2,1,.6,1,.1,1]], #vivir|vivir,farmaco,a #morir|
                  evidence=['Enfermedad', 'Tratamiento','Reaccion'],
                  evidence_card=[2, 2,2])

model.add_cpds(cpd_e, cpd_tr, cpd_r_tr, cpd_f_e_tr_r)

infer = VariableElimination(model)
sampling = BayesianModelSampling(model)

## Inicializando modelo verdadero para simulaciones

In [623]:
true_model_class=true_causal_model(4,infer,sampling,model)


In [624]:
true_model_class.action_simulator()

{'Enfermedad': 1, 'Final': 1, 'Reaccion': 0, 'Tratamiento': 0}

In [625]:
class true_causal_model:
    def __init__(self,N,infer,sampling,model):
        self.num_variables=N
        self.infer=infer
        self.sampling=sampling
        self.true_model=model

    def action_simulator(self):
        response=dict()
        elements = [0,1]
        probabilities = [0.5,0.5]
        res=np.random.choice(elements, 1, p=probabilities)
        enf=np.random.choice(elements, 1, p=probabilities)
        response['Enfermedad']=res[0]
        response['Tratamiento']=enf[0]
        response['Reaccion']=self.infer.map_query(['Reaccion'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad']})['Reaccion']
        response['Final']=self.infer.map_query(['Final'],evidence={'Tratamiento': response['Tratamiento'],'Enfermedad': response['Enfermedad'],'Reaccion':response['Reaccion']})['Final']
        return(response)
        

## Para simular:

In [626]:
class structure_learning:
    def __init__(self, alpha, variables, true_model,model_simulator, orden):
        self.local_model=true_model ## Este es el directamente creado con PGMPY, no la clase true_causal_model
        self.simulator=model_simulator.action_simulator
        self.a=alpha
        self.orden=orden
        self.variables=variables
        self.causas_posibles=list()
        self.beliefs=dict()
    
    def print_variables(self):
        return(self.variables)
    
    def print_orden(self):
        return(self.orden)
    
    def possible_causes(self):
        for i in range(len(self.variables)):
            for j in range(len(self.variables)):
                if(self.variables[i] != self.variables[j]):
                    lista_temp=[self.variables[i],self.variables[j]]
                    if(lista_temp not in self.orden):
                        self.causas_posibles.append(lista_temp)
        return(self.causas_posibles)
    
    
    def initial_beliefs(self):
        self.causas_posibles=self.possible_causes()
        causas_posibles=self.causas_posibles
        for i in causas_posibles:
            i=str(i)
            self.beliefs[i]=np.random.rand(1).tolist()[0]
        return(self.beliefs)

    def observing_external(self):
        datos=self.simulator()
        return(datos)
    
    def belief_updating(self):
        datos=self.observing_external()
        print(datos)
        for i in self.causas_posibles:
            if(datos[i[0]]==1 and datos[i[1]]==1):
                i=str(i)
                self.beliefs[i]=self.beliefs[i]+self.a*(1-self.beliefs[i])
            else:
                if(datos[i[0]]==1 and datos[i[1]]==0):
                    i=str(i)
                    self.beliefs[i]=self.a*self.beliefs[i]
                else:
                    if(datos[i[0]]==0 and datos[i[1]]==1):
                        i=str(i)
                        self.beliefs[i]=self.a*self.beliefs[i]
                   # else:
                   #     if(datos[i[0]]==0 and datos[i[1]]==0):
                            
        return(self.beliefs)
    
            
        
        
        

## Inicializando clase para aprendizaje de estructura

In [627]:
aprendizaje=structure_learning(alpha,variables,model,true_model_class,orden)

In [628]:
aprendizaje.observing_external()

{'Enfermedad': 0, 'Final': 1, 'Reaccion': 1, 'Tratamiento': 1}

In [629]:
aprendizaje.initial_beliefs()

{"['Enfermedad', 'Final']": 0.11247686487073938,
 "['Enfermedad', 'Reaccion']": 0.9545007563520017,
 "['Enfermedad', 'Tratamiento']": 0.5714960415820615,
 "['Reaccion', 'Final']": 0.2959079879922909,
 "['Tratamiento', 'Final']": 0.18607108036487952,
 "['Tratamiento', 'Reaccion']": 0.17242955188227838}

In [639]:
aprendizaje.belief_updating()

{'Enfermedad': 0, 'Tratamiento': 0, 'Reaccion': 0, 'Final': 0}


{"['Enfermedad', 'Final']": 0.9988609271603157,
 "['Enfermedad', 'Reaccion']": 0.7430001563271796,
 "['Enfermedad', 'Tratamiento']": 0.7411066393657317,
 "['Reaccion', 'Final']": 0.7475579538376338,
 "['Tratamiento', 'Final']": 0.7473769481915192,
 "['Tratamiento', 'Reaccion']": 0.9967673029370401}

In [517]:
lista_prueba=list()
causas_posibles=list()

In [518]:
variables
for i in range(len(variables)):
    for j in range(len(variables)):
        if(variables[i] != variables[j]):
            lista_temp=[variables[i],variables[j]]
            if(lista_temp not in orden):
                causas_posibles.append(lista_temp)

In [519]:
causas_posibles

[['Enfermedad', 'Tratamiento'],
 ['Enfermedad', 'Reaccion'],
 ['Enfermedad', 'Final'],
 ['Tratamiento', 'Reaccion'],
 ['Tratamiento', 'Final'],
 ['Reaccion', 'Final']]

In [520]:
beliefs=dict()

In [521]:
for i in causas_posibles:
    i=str(i)
    beliefs[i]=np.random.rand(1).tolist()[0]
    a=0.5

In [523]:
beliefs

{"['Enfermedad', 'Final']": 0.07851783356862196,
 "['Enfermedad', 'Reaccion']": 0.9940126956664775,
 "['Enfermedad', 'Tratamiento']": 0.658307719441343,
 "['Reaccion', 'Final']": 0.41196424432502143,
 "['Tratamiento', 'Final']": 0.931092792873572,
 "['Tratamiento', 'Reaccion']": 0.8587132550163319}

In [207]:
for i in causas_posibles:
    if(i[0]==1 and i[1==1]):
        i=str(i)
        beliefs[i]=beliefs[i]+a*(1-beliefs[i])
    else:
        i=str(i)
        beliefs[i]=a*beliefs[i]

In [516]:
causas_posibles

[]

In [199]:
for i in causas_posibles:
    i=str(i)
    print(beliefs[i][0])

TypeError: 'float' object is not subscriptable

True

In [78]:
if 4!=3:
    print('a')

a


In [98]:
np.random.rand(1).tolist()

[0.1413996337890877]

In [111]:
1==1 and 2==2

True

In [122]:
alpha

0.5

In [204]:
np.random.rand(1).tolist()[0]

0.5581863603408671

In [529]:
for i in causas_posibles:
    if(i[0]==1 and i[1]==1):
        print('1')
        i=str(i)
        beliefs[i]=beliefs[i]+a*(1-beliefs[i])
    elif(i[0]==1 and i[1]==2):
        print('2')
        i=str(i)
        beliefs[i]=a*beliefs[i]
    elif(i[0]==0 and i[1]==1):
        print('3')
        i=str(i)
        beliefs[i]=a*beliefs[i]

In [525]:
beliefs

{"['Enfermedad', 'Final']": 0.07851783356862196,
 "['Enfermedad', 'Reaccion']": 0.9940126956664775,
 "['Enfermedad', 'Tratamiento']": 0.658307719441343,
 "['Reaccion', 'Final']": 0.41196424432502143,
 "['Tratamiento', 'Final']": 0.931092792873572,
 "['Tratamiento', 'Reaccion']": 0.8587132550163319}

In [526]:
a

0.5

In [535]:
datos=aprendizaje.observing_external()

In [536]:
datos

{'Enfermedad': 0, 'Final': 1, 'Reaccion': 1, 'Tratamiento': 0}

In [538]:
causas_posibles

[['Enfermedad', 'Tratamiento'],
 ['Enfermedad', 'Reaccion'],
 ['Enfermedad', 'Final'],
 ['Tratamiento', 'Reaccion'],
 ['Tratamiento', 'Final'],
 ['Reaccion', 'Final']]

In [548]:
for i in causas_posibles:
    if(datos[i[0]]==1 and datos[i[1]]==1):
        print(i)
        print('si')

['Reaccion', 'Final']
si


In [546]:
datos['Enfermedad']

0